In [7]:
#Import required libraries

import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [8]:
#Set constants for training
seed = 543
log_interval = 10
gamma = 0.99

env = gym.make('CartPole-v1')
env.reset()
state_shape = env.observation_space.shape[0]
no_of_actions = env.action_space.n
print(state_shape)
print(no_of_actions)
torch.manual_seed(seed)


4
2


In [9]:
class Policy(nn.Module):
    """
    implements both actor and critic in one model
    """
    def __init__(self,state_shape = 4,action_size = 2 ):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(state_shape, 128)
        # actor's layer
        self.action_head = nn.Linear(128, action_size)
        # action & reward buffer
        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        """
        forward of both actor and critic
        """
        x = F.relu(self.affine1(x))
        # actor: choses action to take from state s_t
        # by returning probability of each action
        action_prob = F.softmax(self.action_head(x), dim=-1)
        return action_prob


In [10]:
model = Policy(state_shape=state_shape, action_size=no_of_actions)
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()
max_len = 10000 # maximum number of iteration for episode to end

In [11]:

def select_action(state):
    # state = torch.from_numpy(state).float()
    # probs = model(state)
    # #print(probs)
    # # create a categorical distribution over the list of probabilities of actions
    # m = Categorical(probs)
    # action = m.sample()
    # model.saved_actions.append(m.log_prob(action))
    # # the action to take (left or right)
    # return action.item()
    
    state = torch.from_numpy(state).float()
    probs = model(state)

    # create a categorical distribution over the list of probabilities of actions
    m = Categorical(probs)

    # and sample an action using the distribution
    action = m.sample()

    # save to action buffer
    model.saved_actions.append(m.log_prob(action))

    # the action to take (left or right)
    return action.item()


def finish_episode():
    # policy_losses = [] # list to save actor (policy) loss
    # # calculate the true value using rewards returned from the environment  
    # R = torch.tensor(Returns)
    # log_prob = model.saved_actions[state_number]
    # policy_losses.append(-torch.mul(R,log_prob))
    # optimizer.zero_grad()
    # # sum up all the values of policy_losses and value_losses
    # loss = torch.stack(policy_losses).sum()
    # #print(loss)
    # #torch.autograd.set_detect_anomaly(True)
    # loss.backward()
    # optimizer.step()

    R = 0
    saved_actions = model.saved_actions
    policy_losses = [] # list to save actor (policy) loss
    returns = [] # list to save the true values

    # calculate the true value using rewards returned from the environment
    for r in model.rewards[::-1]:
        # calculate the discounted value
        R = r + gamma * R
        returns.insert(0, R)

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    for (log_prob), R in zip(saved_actions, returns):

        # calculate actor (policy) loss
        policy_losses.append(-log_prob * R)

    # reset gradients
    optimizer.zero_grad()

    # sum up all the values of policy_losses and value_losses
    loss = torch.stack(policy_losses).sum()

    # perform backprop
    loss.backward()
    optimizer.step()

    # reset rewards and action buffer
    del model.rewards[:]
    del model.saved_actions[:]
    


def train():
    running_reward = 10

    # run infinitely many episodes
    for i_episode in range(2000):

        # reset environment and episode reward
        state, _ = env.reset()
        ep_reward = 0

        # for each episode, only run 9999 steps so that we don't
        # infinite loop while learning
        for t in range(1, 10000):

            # select action from policy
            action = select_action(state)

            # take the action
            state, reward, done, _, _ = env.step(action)

            model.rewards.append(reward)
            ep_reward += reward
            if done:
                break

        # update cumulative reward
        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward

        # perform backprop
        finish_episode()

        # log results
        if i_episode % log_interval == 0:
            print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                  i_episode, ep_reward, running_reward))

        # check if we have "solved" the cart pole problem
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break

In [12]:

train()

Episode 0	Last reward: 9.00	Average reward: 9.95
Episode 10	Last reward: 16.00	Average reward: 16.84
Episode 20	Last reward: 23.00	Average reward: 16.83
Episode 30	Last reward: 169.00	Average reward: 31.25
Episode 40	Last reward: 52.00	Average reward: 44.33
Episode 50	Last reward: 39.00	Average reward: 47.32
Episode 60	Last reward: 25.00	Average reward: 49.76
Episode 70	Last reward: 31.00	Average reward: 43.07
Episode 80	Last reward: 39.00	Average reward: 43.21
Episode 90	Last reward: 36.00	Average reward: 44.75
Episode 100	Last reward: 157.00	Average reward: 52.22
Episode 110	Last reward: 74.00	Average reward: 57.51
Episode 120	Last reward: 221.00	Average reward: 76.73
Episode 130	Last reward: 119.00	Average reward: 278.12
Episode 140	Last reward: 145.00	Average reward: 215.96
Episode 150	Last reward: 418.00	Average reward: 270.28
Solved! Running reward is now 776.4657257782015 and the last episode runs to 9999 time steps!
